# Getting started

> This is an extension of the first notebook, that also uses the allele names from Pombase.

This is supposed to illustrate a bit the task at hand. Let's start by extracting the allele names from the genotypes.

In principle, alleles in the genotype string should be separated by spaces. Some will be wrong, but that's fine for now.

In [238]:
import pandas
import re

# We make a set to store the alleles
allele_names = set([])
data = pandas.read_csv('../data/strains.tsv', sep='\t')

# We force conversion to string, otherwise empty values are parsed as nans (floats)
data['genotype'] = data['genotype'].astype(str)

for genotype in data.genotype:
    # split the genotype by any separator and add the alleles names to the set
    allele_names.update([a.lower() for a in  re.split("\s+",genotype)])



We want now to get an idea of the ways alleles have been stored. Some of the inconsistencies will be the same for many genes, so we will substitute the name of the gene in the allele by the word "GENE"

> The new thing

Now we make a dictionary where the keys are any of the synonyms of the genes, and the return value is the systematic identifier of the gene.

Note that in the 'real script' we would have to take into account that some gene names are a synonym of one gene, and the main gene of another, for example, `cdc7` is both the main name of `SPBC21.06c` and a synonym of `hsk1` (`SPBC776.12c`). Messy messy!

In [239]:
# Read all gene names and identifiers

systematic_ids = set()
gene_names = set()
other = set()
gene_dictionary = dict()


def add_gene_name(gene_name):
    if re.match(r'[a-z]{3}\d+',gene_name) is not None:
        gene_names.add(gene_name)
    elif re.match(r'SP.+\.\d+c?',gene_name) is not None:
        systematic_ids.add(gene_name)
    else:
        other.add(gene_name)

with open('../data/gene_IDs_names.tsv') as ins:
    # First line does not count
    ins.readline()
    for line in ins:
        fields = line.strip().split('\t')
        add_gene_name(fields[0])
        gene_dictionary[fields[0]] = fields[0]
        if len(fields)>1:
            add_gene_name(fields[1])
            gene_dictionary[fields[1]] = fields[0]
            if len(fields)>2:
                if ',' in fields[2]:
                    [add_gene_name(f) for f in fields[2].split(',')]
                    for f in fields[2].split(','):
                        add_gene_name(f)
                        gene_dictionary[f] = fields[0]
                else:
                    add_gene_name(fields[2])
                    gene_dictionary[fields[2]] = fields[0]

# There are some gene synonyms that fall out of the naming conventions. For now we can leave them outside
# print(other)


> This is also new

Now we create a dictionary in which the keys are the systematic ids, and the values are lists of alleles. We omit the genes that have delta in them, since people typically don't type `delta` (they use the symbol)

In [240]:
allele_dictionary = dict()

with open('../data/alleles_pombemine.tsv') as ins:
    for line in ins:
        ls = line.strip().split('\t')
        if 'delta' not in ls[2]:
            # Check if the key already exists, if not create a list with that value
            systematic_id = ls[0]
            allele_name = ls[2]
            if systematic_id in allele_dictionary:
                allele_dictionary[systematic_id].append(allele_name)
            # Otherwise, append to the existing list
            else:
                allele_dictionary[systematic_id] = [allele_name]

# All lists of alleles should be order in inverse order of length, so that you try to subsitute the longest names first,
# for instance, you should try to replace cdc2-12 before cdc2-1

for key in allele_dictionary:
    allele_dictionary[key].sort(key=len,reverse=True)



Now let's replace the gene names in the allele names by `GENE` and store them in a list. Then let's see what are the most common occurrences in the list.

> New thing

Now, if we find an allele name, we replace by `ALLELE`

In [241]:
alleles_with_replaced_name = list()

for genotype_allele in allele_names:

    for name in re.findall(r'[a-z]{3}\d+',genotype_allele):
        if name in gene_names:

            # Get the systematic id of the gene
            systematic_id = gene_dictionary[name]

            # Find the alleles of that gene and see if any of them is in the alelle name
            allele_found = False
            if systematic_id in allele_dictionary:
                for published_allele in allele_dictionary[systematic_id]:
                    if published_allele.lower() in genotype_allele:
                        genotype_allele = genotype_allele.replace(published_allele.lower(),'ALLELE')
                        allele_found = True
                        break

            # If the allele name was not found, replace with GENE
            if not allele_found:
                genotype_allele = genotype_allele.replace(name,'GENE')

    # Here no caps, because we have changed all genotypes to no caps
    for systematic_id in re.findall(r'sp.+\.\d+c?',genotype_allele):
        if systematic_id in map(str.lower, systematic_ids):
            allele_found = False
            if systematic_id in allele_dictionary:
                for published_allele in allele_dictionary[systematic_id]:
                    if published_allele.lower() in genotype_allele:
                        genotype_allele = genotype_allele.replace(published_allele.lower(),'ALLELE')
                        allele_found = True
                        break

            # If the allele name was not found, replace with GENE
            if not allele_found:
                genotype_allele = genotype_allele.replace(systematic_id,'GENE')

    alleles_with_replaced_name.append(genotype_allele)


We can also replace the resistance markers for `MARKER`

In [242]:
markers = ['kanr','kanmx6','kanmx4','kanmx','hygr','hyg','hphmx','hphr','hph','natmx','natr','nat','kan','natmx6',r'\d*myc',r'\d*flag\d*']

for i in range(len(alleles_with_replaced_name)):
    for marker in markers:
        alleles_with_replaced_name[i] = re.sub(marker,'MARKER',alleles_with_replaced_name[i])

tags = ['tdtomato','megfp','egfp','gfp','mcherry','cfp','spmneongreen','mneongreen','2xyfp','myfp','yfp']

for i in range(len(alleles_with_replaced_name)):
    for tag in tags:
        alleles_with_replaced_name[i] = re.sub(tag,'TAG',alleles_with_replaced_name[i])

# TODO: ask about adh promoter to authors, also prep81 and so probably wrongly used
promoters = [r'p?nmt\d*',r'p?adh\d*',r'prep\d*']

for i in range(len(alleles_with_replaced_name)):
    for promoter in promoters:
        alleles_with_replaced_name[i] = re.sub(promoter,'PROMOTER',alleles_with_replaced_name[i])

# Ignore
# truncations = [r'\d+-\d+']
# trunc_ls = list()

# for i in range(len(alleles_with_replaced_name)):
#     for truncation in truncations:
#         tt = re.findall(truncation,alleles_with_replaced_name[i])
#         alleles_with_replaced_name[i] = re.sub(truncation,'TRUNCATION',alleles_with_replaced_name[i])
#         if len(tt)!=0:
#             trunc_ls.append(alleles_with_replaced_name[i])

# counted = Counter(trunc_ls)
# print(counted.most_common())


We can also replace all spaces for a single dash and leading or trailing symbols

In [243]:
for i in range(len(alleles_with_replaced_name)):
    alleles_with_replaced_name[i] = re.sub(r'[:-<\.]+','-',alleles_with_replaced_name[i])
    alleles_with_replaced_name[i] = re.sub(r'^-','',alleles_with_replaced_name[i])
    alleles_with_replaced_name[i] = re.sub(r'-$','',alleles_with_replaced_name[i])


In [244]:
from collections import Counter

counted = Counter(alleles_with_replaced_name)
# Sort
result = counted.most_common()

# Write into file

with open('dummy00.txt','w') as out:
    for r in result:
        out.write(f'{r[0]} {r[1]}\n')

print('reduced by',1-len(result)/len(allele_names))

reduced by 0.5252461951656222


# Further refinement

We can now look the most common occurrences of words of N letters, that is a good way to fish for patterns

In [245]:
def count_most_common_consecutive_N_characters(file,n):
    all_occurrences = list()
    with open(file,'r') as ins:
        for line in ins:
            line = line.strip()
            line = line.replace('GENE','')
            line = line.replace('MARKER','')
            line = line.replace('TAG','')
            line = line.replace('ALLELE','')
            line = line.replace('PROMOTER','')
            if len(line)>=n:
                all_occurrences += [line[i:i+n] for i in range(0,len(line)-n+1)]

    return Counter(all_occurrences)

counted = count_most_common_consecutive_N_characters('dummy00.txt',6)

with open('dummy01.txt','w') as out:
    for r in counted.most_common():
        out.write(f'{r[0]} {r[1]}\n')


